# *M5 Submission file summary 

1) This leaderboard is calculated with  approximately 50% of the test data. 

Test_data period ... ? 


Validation ( Public leaderboard )
Evluation ( Private leaderboard )

Validation d_1914 - d_1941
Evaluation d_1942 - d_1969 (28 days) (test period)


sales_train_validation.csv [d_1 - d_1913]
sales_train_evaluation.csv [d_1 - d_1941]


test period in calendar file = [d_1942 - d_1969] 29 days 



Pulbic 
Validation d_1914 - d_1941

Private
Evaluation d_1942 - d_1969 (28 days) (test period)





In [1]:
import pandas as pd

In [2]:
sales_validation = pd.read_csv("C:\Eric\M5_Data\sales_train_validation.csv")

In [3]:
sales_evaluation = pd.read_csv("C:\Eric\M5_Data\sales_train_evaluation.csv")

In [10]:
sales_validation.shape

(30490, 1919)

In [13]:
sales_evaluation.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0
30489,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,2,0,5,4,0,2,2,5,1


In [4]:
calendar = pd.read_csv("C:\Eric\M5_Data\calendar.csv")

In [15]:
calendar.shape

(1969, 14)

In [17]:
calendar.tail()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [ ]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]

train_cols = sales_validation.columns[~sales_validation.columns.isin(useless_cols)]

# point 
X_train = sales_validation[train_cols]
y_train = sales_validation["sales"]


In [ ]:
np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)

#
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)




train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)

fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!



In [7]:
sales_validation.index.values

array([    0,     1,     2, ..., 30487, 30488, 30489], dtype=int64)

In [ ]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [ ]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

In [ ]:
# memo 
# 3. enumerate
# 반복문 사용 시 몇 번째 반복문인지 확인이 필요할 수 있습니다. 이때 사용합니다.
# 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환합니다.
# >>> t = [1, 5, 7, 33, 39, 52]

In [9]:
alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):
    print(icount, (alpha, weight))

0 (1.028, 0.3333333333333333)
1 (1.023, 0.3333333333333333)
2 (1.018, 0.3333333333333333)


In [ ]:
for icount, 

In [13]:
[1/len(alphas)]*len(alphas)
#*len(alphas)

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

In [17]:
cols = [f"F{i}" for i in range(1,29)]
print(cols)

['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']


In [36]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [25]:
calendar.tail()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [28]:
whos

Variable           Type         Data/Info
-----------------------------------------
alpha              float        1.018
alphas             list         n=3
calendar           DataFrame                date  wm_yr_w<...>n[1969 rows x 14 columns]
cols               list         n=28
datetime           module       <module 'datetime' from '<...>onda3\\lib\\datetime.py'>
day                datetime     2016-05-22 00:00:00
fday               datetime     2016-04-25 00:00:00
gc                 module       <module 'gc' (built-in)>
icount             int          2
lgb                module       <module 'lightgbm' from '<...>\\lightgbm\\__init__.py'>
now                datetime     2020-06-25 14:30:29.203580
np                 module       <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
pd                 module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
sales_evaluation   DataFrame                             <...>0490 rows x 1947 columns]
sales_validation   Data

In [29]:
sales_validation.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1919)

In [46]:
import datetime
for tdelta in range(0, 28):        
    day = fday + timedelta(days=tdelta)
    print(tdelta, day)

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00


In [52]:
calendar.tail()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-15,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-15,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-15,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-15,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [51]:
calendar['date'] ="2016-06-15"

In [45]:
max_lags = 57 

fday = datetime(2016,4, 25) 

for tdelta in range(0, 28):        
    day = fday + timedelta(days=tdelta)
# print(tdelta, day)
    
    print(day - timedelta(days=max_lags))
    tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()

    create_fea(tst)
    tst = tst.loc[tst.date == day , train_cols]
    te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev
        
        
    
    
# day = fday + timedelta(days=tdelta)


# day - timedelta(days=max_lags)
# 2월 28일부터, 3월 26일까지 

# day = fday + timedelta(days=tdelta)
# 4월 25일부터, 5월 22일까지 

# day - timedelta(days=max_lags)#2월28일부터, 3월 26일까지 <= te.date <= day #(0425~0522)


#d1885 ~ d1913 #4월 한달 간의 기간을 의미함 

te.date 

2016-02-28 00:00:00
2016-02-29 00:00:00
2016-03-01 00:00:00
2016-03-02 00:00:00
2016-03-03 00:00:00
2016-03-04 00:00:00
2016-03-05 00:00:00
2016-03-06 00:00:00
2016-03-07 00:00:00
2016-03-08 00:00:00
2016-03-09 00:00:00
2016-03-10 00:00:00
2016-03-11 00:00:00
2016-03-12 00:00:00
2016-03-13 00:00:00
2016-03-14 00:00:00
2016-03-15 00:00:00
2016-03-16 00:00:00
2016-03-17 00:00:00
2016-03-18 00:00:00
2016-03-19 00:00:00
2016-03-20 00:00:00
2016-03-21 00:00:00
2016-03-22 00:00:00
2016-03-23 00:00:00
2016-03-24 00:00:00
2016-03-25 00:00:00
2016-03-26 00:00:00


In [43]:

max_lags = 57 
a = timedelta(days=max_lags)
print(a)



57 days, 0:00:00


In [21]:
fday = datetime(2016,4, 25) d_1914
max_lags = 57 

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.


for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]
    
    for tdelta in range(0, 28):
        
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        
# day - timedelta(days=max_lags) <= te.date <= day (0425~0522)
         
        create_fea(tst)
       
    
        tst = tst.loc[tst.date == day , train_cols]
        
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev

# 그렇다면 날짜는 알아냈고.. predict 에 넣은건 왜 dat 에 해당되는 날짜를 넣은거지 ?         
        
        
    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
    
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00


In [18]:
_

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [27]:
# timedelta sample 

import datetime

now = datetime.datetime.now()
print(now)

tomorrow = now+datetime.timedelta(days=1)
print(tomorrow)

2020-06-25 14:30:29.203580
2020-06-26 14:30:29.203580
